This file is part of GaPSE
Copyright (C) 2022 Matteo Foglieni

GaPSE is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

GaPSE is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU
General Public License for more details.

You should have received a copy of the GNU General Public License
along with GaPSE. If not, see <http://www.gnu.org/licenses/>.
```


```

# Generic Window

This is a small tutorial in order to show how to use GaPSE when one is interested in the power spectrum obtained with a generic window function, and not with the toy-model we implemented.
The theoretical formula we will use is the following (Eq.(2.13) of Castorina, Di Dio (2021)); we remember that all the distances $s, s_1, s_2$ are comoving distances measured in $h_0^{-1}\mathrm{Mpc}$:

$$
    \left\langle \hat{P}_L(k) \right\rangle = 
        \frac{2 L + 1}{A} (-i)^L
        \sum_{\ell=0}^{\infty} 
        \sum_{\ell_1=0}^{\infty} 
        \begin{pmatrix}
            L & \ell & \ell_1 \\
            0 & 0 & 0
        \end{pmatrix}^2
        \int_0^{\infty}\!\mathrm{d} s \, s^2 \, \xi_\ell(s, s_{\rm eff}) \, 
        j_L(k s) \, Q_{\ell_1}(s) \, ,
$$

where:
- $\left\langle \hat{P}_L(k) \right\rangle$ is the order $L$ Power Spectrum of the effect we are interested in; we are basing this expression on the Yamamoto estimator (see Yamamoto (2000) and Yamamoto (2006))
- $A$ is a normalization constant
- the 2x3 matrix represents the Wigner-3j symbols
- $\xi_\ell$ is the order $\ell$ multipole of the Two-Point Correlation Function (TPCF)
- $j_L$ is the spherical Bessel function of order $L$
- $s_{\mathrm{eff}}$ is the comoving distance associated with the effective redshift (see the `TUTORIAL.ipynb` notebook)

$Q_{\ell_1}$ can be easily estimated with FFT methods:

$$
\begin{split}
    Q_{\ell_1}(s) &:= \int_0^{\infty}\mathrm{d}s_1 \; s_1^2 \;
    \phi(s_1) \; F_{\ell_1}(s_1, s) \\
    F_{\ell_1} (s_1 , s) &:= 
    \int_{4\pi} \mathrm{d}\Omega_{\mathbf{\hat{s}}} \,  
    \int_{4\pi} \mathrm{d}\Omega_{\mathbf{\hat{s}}_1} \,
    \phi(s_2) \, W(\mathbf{\hat{s}}_1) \, W(\mathbf{\hat{s}}_2) \,
    \mathcal{L}_{\ell_1}(\mathbf{\hat{s}} \cdot \mathbf{\hat{s}}_1)  \, .
\end{split}
$$

where:
- $\mathcal{L}_{\ell_1}$ is the Legendre polynomial of order $\ell_1$
- $\mathrm{d}\Omega_{\mathbf{\hat{s}}}$ is the infinitesimal solid angle pointing in the direction of the versor $\mathbf{\hat{s}}$
- $\phi(s)$ and  $W(\mathbf{\hat{s}})$ are respectively the radial and angular part of your window function (we remember that we assumed that such separability of the window function is possible)

Suppose we want the Power Spectrum of the Galaxy Number Counts (GNC) General Relatvistic (GR) effects. The idea is simple:
- first of all, you must compute the $Q_{\ell_1}(s)$ multipoles ($0 \leq \ell_1 \leq 4$ should be enough in almost all the realistic cases, but depending on your specific choise maybe you could even need less than that) and store them in a file let's call it `Ql_multipoles.txt`.  The file should have the following structure:
```text
    $ cat Ql_multipoles.txt 
    # Any comment line in the file must start with a #
    # you can have how many comment lines you want in the header; they 
    # will be all skipped.
    # Then you must provide in blank space separated columns:
    # - as first column, the comoving distance values, measured in Mpc/h_0
    # - from the second column onwards, all the Q_{\ell_1} multipoles you want;
    #   they must be ordered followinf the ascending multipole order (so \ell_1 = 0
    #   must be the 2 column), and you can go as further as you want in the multipole
    #   order
    # 
    # s [Mpc/h_0] 	 Q_{l1=0} 	 Q_{l1=1} 	 Q_{l1=2} 	 ...
    1.0 	0.9999999999999999 	 1.445269850978461e-7 	 0.000011917268941324522	...
    21.0	0.9832914433168294 	 -0.0025537781362117177  -0.0033199998619560947	    ...
    41.0	0.9669175943095181 	 -0.004923364937797496 	 -0.006463561496567318	    ... 	
    ... 	... 				 ... 					 ...
```

- then you can proceed in the computation of all the GNC Two-Point Correlation Functions (TPCFs) for the same multipole orders, setting `use_windows=false`
- finally, you'll use the functions `create_file_for_XiMultipole` and `PS_multipole_GenWin` , and you'll get the PS multipole described in the previous equation.

In [8]:
DIR = "Generic_Window/";
COMPUTE_XIS_GNC = false;

## Creation of a Cosmology

As always, let's create a Cosmology struct. Note that, even if you want to study your window function and you are not interested in the toy-model one, it's (currently) mandatory to provide the files for a WindowF and a WindowFIntegrated anyway. Just give the default ones in the `Data` directory (even if are suited for a different redshift bin, it does not matter).

Just remember to ALWAYS SET `use_windows = false` in each TPCF computation! 

In [1]:
PATH_TO_GAPSE = "../";

In [2]:
include(PATH_TO_GAPSE * "src/GaPSE.jl");

In [3]:
using DelimitedFiles, Plots, Dierckx, SpecialFunctions, TwoFAST, ProgressMeter
using Printf, LaTeXStrings, GridInterpolations, LegendrePolynomials

In [4]:
#FILE_F_MAP =  PATH_TO_GAPSE * "data/NEW_F_pi2.txt";
FILE_F_MAP =  PATH_TO_GAPSE * "data/F_REFERENCE_pi2.txt";
#=
kwargs_map_F_hcub = Dict(
     :θ_max => π / 2.0, :tolerance => 1e-10, 
     :rtol => 1e-2, :atol => 1e-3, :pr => true,
);

kwargs_map_F_trap = Dict(
     :θ_max => π / 2.0, :tolerance => 1e-10, 
     :N => 1000, :pr => true,
);

xs = [x for x in 0:0.02:5]
μs = union(
    [μ for μ in range(-1.0, -0.98, length = 50)], 
    [μ for μ in range(-0.98, 0.98, length = 102)],
    [μ for μ in range(0.98, 1.0, length = 50)]);
GaPSE.print_map_F(FILE_F_MAP, xs, μs; 
    alg = :trap, Fmap_opts = kwargs_map_F_trap # we recommend to use :trap
    #alg = :hcub, Fmap_opts = kwargs_map_F_hcub # but you can use also :hcub if you prefer
)
=#

In [5]:
z_min, z_max, θ_max = 1.0, 1.5, π / 2.0

FILE_BACKGROUND = PATH_TO_GAPSE * "data/WideA_ZA_background.dat";

#=
WFI_opts = Dict(
    :llim => nothing, :rlim => nothing, 
    :rtol => 1e-2, :atol => 0.0, 
    :N => 1000, :pr => true,
)
=#

params = GaPSE.CosmoParams(z_min, z_max, θ_max;
    Ω_b = 0.0489, Ω_cdm = 0.251020, h_0 = 0.70, s_lim=1e-2,
    s_b = 0.0, 𝑓_evo = 0.0, b = 1.5,
    IPS_opts = Dict(
        :fit_left_min => 1e-6, :fit_left_max => 3e-6,
        :fit_right_min => 1e1, :fit_right_max => 2e1),
    IPSTools_opts = Dict(
        :N => 1024, :fit_min => 0.05, :fit_max => 0.5, 
        :con => true, :k_min => 1e-8, :k_max => 10.0),
    #=
    WFI_opts = Dict(
        :llim => nothing, :rlim => nothing, 
        :rtol => 1e-2, :atol => 0.0, 
        :N => 1000, :pr => true,)
    =#
);

In [6]:
#FILE_F_MAP =  PATH_TO_GAPSE * "data/NEW_F_pi2.txt";
#FILE_IF_MAP = PATH_TO_GAPSE * "data/NEW_IntegrF_pi2_z115.txt";

#FILE_F_MAP = PATH_TO_GAPSE*"data/F_REFERENCE_pi2.txt";
FILE_IF_MAP = PATH_TO_GAPSE*"data/IntegrF_REFERENCE_pi2_z115.txt";

#=
calc_μs = union(
    [μ for μ in range(-1.0, -0.98, length = 50)], 
    [μ for μ in range(-0.98, 0.98, length = 102)],
    [μ for μ in range(0.98, 1.0, length = 50)]);

GaPSE.print_map_IntegratedF(
    z_min, z_max, calc_μs,
    FILE_F_MAP, FILE_IF_MAP, 
    FILE_BACKGROUND;
    alg = :trap, N_ss = 200, m = 2.1,
    Dict(
        :llim => nothing, :rlim => nothing, 
        :rtol => 1e-2, :atol => 0.0, 
        :N => 1000, :pr => true,
    )...
)
=#

In [7]:
FILE_PS = PATH_TO_GAPSE * "test/datatest/file_pk.txt";
cosmo = GaPSE.Cosmology(params, FILE_BACKGROUND, FILE_PS, FILE_F_MAP, FILE_IF_MAP);

GaPSE.parameters_used(stdout, cosmo)

###############
#    GaPSE    #
############### 
#
# The Cosmology considered had the following paremeters:
# 
# - Matter Power Spectrum input file: "../test/datatest/file_pk.txt"
# - Background data input file: "../data/WideA_ZA_background.dat"
# - F window function input file: "../data/F_REFERENCE_pi2.txt"
# - Integrated F window function input file: "../data/IntegrF_REFERENCE_pi2_z115.txt"
#
# - Basic CosmoParams considered: 
#	 z_min = 1.0 	 z_max = 1.5
#	 θ_max = 1.5707963267948966 [rad] 	 h_0 = 0.7
#	 Ω_b = 0.0489 	 Ω_cdm = 0.25102 	 Ω_M0 = 0.29992
#	 b = 1.5 	 f_evo = 0.0 	 s_b = 0.0
#
# - CosmoParams about the Input Power Spectrum: 
#	 fit_left_min = 1.0e-6 	 fit_right_min = 10.0 	 
#	 fit_left_max = 3.0e-6 	 fit_right_max = 20.0 	 
#
# - CosmoParams about the Input Power Spectrum Tools: 
#	 k_max = 10.0 	 fit_max = 0.5 	 N = 1024 	 
#	 con = true 	 k_min = 1.0e-8 	 fit_min = 0.05 	 
#
# - Computed quantities: 
# 	 effective redshift z_eff = 1.2613556177172358 
# 	 comoving s_m

## Ql multipoles

For this tutorial, we will consider again as window function the azimutally simmetry toy-model one described in the `TUTORIAL.ipynb`. 
However, we will not exploit its peculiar simmetry to get a simplified analitical expression; we will instead use the general Power Spectrum equation described at the beginning of this notebook. 

In order to do so, the only equation we miss is the one relating the window function $F(x,\mu)$ (that we can compute with the GaPSE functions) and the multipoles $F_{\ell_1}$. Having a look at Eq.(A.11-12) of Castorina, Di Dio (2021), we have: 

$$
\begin{split}
    F(s_1, s_ \mu) :&= \phi\left(\sqrt{s_1^2 + s^2 + 2 \, s_1 \, s \, \mu}\right) \; F\left(\frac{s}{s_1}, \mu\right) \\
                    &= \sum_{l_1=0}^{\infty} \frac{2 \ell_1 + 1}{2} \, \mathcal{L}_{\ell_1}(\mu) \, F_{\ell_1}(s_1,s)
\end{split}
$$

$$
\Rightarrow \quad F_{\ell_1}(s_1,s) = \int_{-1}^{+1}\mathrm{d}\mu \; \mathcal{L}_{\ell_1}(\mu) \; 
    \phi\left(\sqrt{s_1^2 + s^2 + 2 \, s_1 \, s \, \mu}\right) \; F\left(\frac{s}{s_1}, \mu\right) 
$$

All the analitical formulae necessary to compute $Q_{\ell_1}$ from $F(x,\mu)$ are written down; we must make the integral over $\mathrm{d}\mu$ first (with $\mathcal{L}_{\ell_1}(\mu) \; \phi\left(\sqrt{s_1^2 + s^2 + 2 \, s_1 \, s \, \mu}\right)$ as weight) and then the one over $\mathrm{d}s_1$ (with $s_1^2 \; \phi(s_1) $ as weight).

We have such $Q_{\ell_1}$ multipoles already stored in the file `data/Ql_multipoles.txt`, until $L=4$.

## Compute the TPCFs

 Here we give the code in order to compute all the TPCFs for the GNC group, without the observer velocity terms, until $L=4$.

In [ ]:
name_xis_GNC_L0_noF_noobsvel_file = DIR * "xis_GNC_L0_noF_noobsvel.txt";
name_xis_GNC_L1_noF_noobsvel_file = DIR * "xis_GNC_L1_noF_noobsvel.txt";
name_xis_GNC_L2_noF_noobsvel_file = DIR * "xis_GNC_L2_noF_noobsvel.txt";
name_xis_GNC_L3_noF_noobsvel_file = DIR * "xis_GNC_L3_noF_noobsvel.txt";
name_xis_GNC_L4_noF_noobsvel_file = DIR * "xis_GNC_L4_noF_noobsvel.txt";

In [ ]:
if COMPUTE_XIS_GNC == true
    GaPSE.print_map_sum_ξ_GNC_multipole(
        cosmo, name_xis_GNC_L0_noF_noobsvel_file,
        10 .^ range(0, log10(2 * cosmo.s_max), length=500);
        use_windows=false, L=0, alg=:quad, obs=:noobsvel,
        single=true, enhancer=1e8,
        N_trap=200, N_lob=200, atol_quad=0.0, rtol_quad=1e-2,
        N_χs=100, N_χs_2=60
    )
end

In [ ]:
if COMPUTE_XIS_GNC == true
    GaPSE.print_map_sum_ξ_GNC_multipole(
        cosmo, name_xis_GNC_L1_noF_noobsvel_file,
        10 .^ range(0, log10(2 * cosmo.s_max), length=500);
        use_windows=false, L=1, alg=:lobatto, obs=:noobsvel,
        single=true, enhancer=1e8,
        N_trap=200, N_lob=1000, atol_quad=0.0, rtol_quad=1e-2,
        N_χs=100, N_χs_2=50
    )
end

In [ ]:
if COMPUTE_XIS_GNC == true
    GaPSE.print_map_sum_ξ_GNC_multipole(
        cosmo, name_xis_GNC_L2_noF_noobsvel_file,
        10 .^ range(0, log10(2 * cosmo.s_max), length=500);
        use_windows=false, L=2, alg=:quad, obs=:noobsvel,
        single=true, enhancer=1e8,
        N_trap=200, N_lob=1000, atol_quad=0.0, rtol_quad=1e-2,
        N_χs=100, N_χs_2=50
    )
end

In [ ]:
if COMPUTE_XIS_GNC == true
    GaPSE.print_map_sum_ξ_GNC_multipole(
        cosmo, name_xis_GNC_L3_noF_noobsvel_file,
        10 .^ range(0, log10(2 * cosmo.s_max), length=500);
        use_windows=false, L=3, alg=:lobatto, obs=:noobsvel,
        single=true, enhancer=1e8,
        N_trap=200, N_lob=1000, atol_quad=0.0, rtol_quad=1e-2,
        N_χs=100, N_χs_2=50
    )
end

In [ ]:
if COMPUTE_XIS_GNC == true
    GaPSE.print_map_sum_ξ_GNC_multipole(
        cosmo, name_xis_GNC_L4_noF_noobsvel_file,
        10 .^ range(0, log10(2 * cosmo.s_max), length=500);
        use_windows=false, L=4, alg=:lobatto, obs=:noobsvel,
        single=true, enhancer=1e8,
        N_trap=200, N_lob=1000, atol_quad=0.0, rtol_quad=1e-2,
        N_χs=100, N_χs_2=50
    )
end

## 